In [2]:
import pandas as pd
import numpy as np
import seaborn
%pylab notebook

import hashlib
import humanhash

Populating the interactive namespace from numpy and matplotlib


# Tablas Pivote

Es muy común la agrupación de información en formato 'stack' donde tenemos filas de datos que demuestran una correlación entre dos sets de valores.

Las tablas pivote son una forma de re-ordenar los datos en una estructura tabular donde podemos agrupar los valores convirtiendo las tuplas entre valores numéricos.

En este caso de ejemplo, crearemos un dataset de prueba con la diversidad de restaurantes en varias ciudades centroamericanas. En esta vamos a comenzar a trabajar con dos columnas, una donde describe cada ciudad y la otra con la variedad de cocina disponible en cada una.

In [3]:
data_restaurantes = {
    'ciudades': ['Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','Guatemala','San José','San José','San José','San José','San José','San Salvador','San Salvador','San Salvador'],
    'culinaria': ['Chapina','Chapina','China','Thai','Italiana','Chapina','Italiana','China','Tica','Chapina','Tica','Tica','Italiana','China']
}

restaurantes_dataframe_pares = pd.DataFrame(data_restaurantes)
restaurantes_dataframe_pares

,ciudades,culinaria
0,Guatemala,Chapina
1,Guatemala,Chapina
2,Guatemala,China
3,Guatemala,Thai
4,Guatemala,Italiana
5,Guatemala,Chapina
6,San José,Italiana
7,San José,China
8,San José,Tica
9,San José,Chapina


Podemos ver entonces este listado de valores, tupla por tupla. Que tal si queremos contar la presencia de cada tipo de cocina en cada región. Probemos utilizando entonces el comando [DataFrame.pivot_table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html) de Pandas.

Este pide unos cuantos argumentos los que podemos ver en la documentación. Unos cuantos son obvios, como el definir las filas y columnas que esperamos de la tabla objetivo.
Sin embargo, lo más notable es que ya que los tipos de datos que estamos utilizando no son numéricos, es necesario que definamos una funcion de agrupación que nos permita contar la cantidad de instancias de cada combinación.

In [4]:
def funcion_agrupacion(elemento):
    return True

agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"],
    columns="culinaria",
    aggfunc=funcion_agrupacion,
    fill_value=False)
agrupacion_culinaria

culinaria,Chapina,China,Italiana,Thai,Tica
ciudades,,,,,
Guatemala,True,True,True,True,False
San José,True,True,True,False,True
San Salvador,False,True,True,False,True


Hmm, esto ya se ve con la forma que queremos, sin embargo solo nos muestra la presencia o ausencia de algún tipo de cocina. Esto es fácil de explicar ya que definimos nuestra función de agrupación como retornar True si existe presencia a retornar False si No. ¿Qué tal si hacemos una mejor función de agrupación?

In [5]:
def funcion_agrupacion(elemento):
    '''Contemos cuantas instancias de cada tupla existen.'''
    ## El comando len(iterable) cuenta la cantidad de elementos que tiene el objeto iterable que le pasemos
    ## los elementos iterables pueden ser listas normales, Series de NumPy o Pandas, o diccionarios y otros tipos de datos.
    return len(elemento)
agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(index=["ciudades"], columns="culinaria", aggfunc=lambda x: funcion_agrupacion(x), fill_value=0)
agrupacion_culinaria

culinaria,Chapina,China,Italiana,Thai,Tica
ciudades,,,,,
Guatemala,3,1,1,1,0
San José,1,1,1,0,2
San Salvador,0,1,1,0,1


¡Genial! Ahora ya tenemos agrupadas estas de una forma coherente. Ahora ya podemos seguir manipulando y editando estos datos.
Pero, ¿qué tal si hacemos un poco más simple esta llamada? Al final nuestra función de agrupación lo unico que hace es contar la cantidad de tuplas sobre las cuales aplica el pivote. ¿Qué tal si lo hacemos un poco más simple?

In [6]:
agrupacion_culinaria = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"], 
    columns="culinaria", 
    aggfunc=len, ## Enviamos directamente la función de agrupación. Entre otras funcionas de agrupacíon útiles está np.sum (la función de suma de NumPy) y np.mean (media)
    fill_value=0)
agrupacion_culinaria

culinaria,Chapina,China,Italiana,Thai,Tica
ciudades,,,,,
Guatemala,3,1,1,1,0
San José,1,1,1,0,2
San Salvador,0,1,1,0,1


Claro, la tabla resultante se comporta exactamente igual y tiene todas las propiedades nativas de los DataFrames. ¿Qué tal si limitamos la query a solo los lugares en ciudad de Guatemala?

In [7]:
agrupacion_culinaria.query('ciudades == ["Guatemala"]')

culinaria,Chapina,China,Italiana,Thai,Tica
ciudades,,,,,
Guatemala,3,1,1,1,0


In [8]:
help(restaurantes_dataframe_pares.pivot_table)

Help on method pivot_table in module pandas.core.reshape.pivot:

pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All') method of pandas.core.frame.DataFrame instance
    Create a spreadsheet-style pivot table as a DataFrame. The levels in the
    pivot table will be stored in MultiIndex objects (hierarchical indexes) on
    the index and columns of the result DataFrame
    
    Parameters
    ----------
    data : DataFrame
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The list
        can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed, it
        is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same len

In [9]:
## Con el argumento Margins, Panda calcula los valores sumados de los totales por agrupación.
agrupacion_culinaria_m = restaurantes_dataframe_pares.pivot_table(
    index=["ciudades"],
    columns="culinaria",
    aggfunc=len,
    fill_value=0,
    margins=True,
    margins_name="Total")
agrupacion_culinaria_m

culinaria,Chapina,China,Italiana,Thai,Tica,Total
ciudades,,,,,,
Guatemala,3.0,1.0,1.0,1.0,0.0,6.0
San José,1.0,1.0,1.0,0.0,2.0,5.0
San Salvador,0.0,1.0,1.0,0.0,1.0,3.0
Total,4.0,3.0,3.0,1.0,3.0,14.0


## Múltiples Valores y Múltiples Indices
¿Qué tal si tenemos datos que tienen una estructura Jerárquica inherente? Podemos utilizar la misma forma de multi indexación que vimos en el seminario pasado, lo importante es que a la hora de la definición del índice, Pandas es capaz de manipularlos e inteligentemente ordenar los niveles acorde.

In [10]:
restaurantes_dataframe_pares['estrellas'] = [5,3,3,5,3,1,2,2,4,3,4,3,2,3]
restaurantes_dataframe_estrellas = restaurantes_dataframe_pares
restaurantes_dataframe_estrellas

,ciudades,culinaria,estrellas
0,Guatemala,Chapina,5
1,Guatemala,Chapina,3
2,Guatemala,China,3
3,Guatemala,Thai,5
4,Guatemala,Italiana,3
5,Guatemala,Chapina,1
6,San José,Italiana,2
7,San José,China,2
8,San José,Tica,4
9,San José,Chapina,3


Pivotando sobre la especialidad culinaria y estrellas, podemos las ciudades con la mayor oferta culinaria, o cuales tienen el mejor promedio de estrellas.

In [11]:
agrupacion_culinaria_promedio_estrellas = restaurantes_dataframe_estrellas.pivot_table(
    index=["ciudades"], 
    values=["culinaria", "estrellas"], 
    aggfunc={"culinaria":len,"estrellas":np.mean},
    fill_value=0)
agrupacion_culinaria_promedio_estrellas

,culinaria,estrellas
ciudades,,
Guatemala,6,3.333333
San José,5,3.000000
San Salvador,3,2.666667


¿Qué tal si queremos ver cuantas estrellas en promedio tienen los restaurantes, por clase de comida, por ciudad?

In [12]:
agrupacion_culinaria_por_estrellas = restaurantes_dataframe_estrellas.pivot_table(
    index=["culinaria"], 
    values=["estrellas"], 
    columns=["estrellas"],
    aggfunc={"estrellas":np.mean},
    fill_value=0)
agrupacion_culinaria_por_estrellas

estrellas            
estrellas         1  2  3  4  5
culinaria                      
Chapina           1  0  3  0  5
China             0  2  3  0  0
Italiana          0  2  3  0  0
Thai              0  0  0  0  5
Tica              0  0  3  4  0

In [13]:
agrupacion_culinaria_por_estrellas = restaurantes_dataframe_estrellas.pivot_table(
    index=["culinaria"], 
    values=["estrellas"], 
    columns=["estrellas"],
    aggfunc={"estrellas":len},
    fill_value=0)
agrupacion_culinaria_por_estrellas

estrellas            
estrellas         1  2  3  4  5
culinaria                      
Chapina           1  0  2  0  1
China             0  1  2  0  0
Italiana          0  2  1  0  0
Thai              0  0  0  0  1
Tica              0  0  1  2  0

Hmm... esto no es muy útil, solo nos dice tautológicamente, que los restaurantes de 'n' estrellas tienen 'n' estrellas. ¿Pueden ver porque el error?

In [14]:
agrupacion_culinaria_por_estrellas = restaurantes_dataframe_estrellas.pivot_table(
    index=["ciudades"], 
    values=["estrellas"], 
    columns=["culinaria"], ## Aqui es obvio ver que lo que queremos es diferenciar por variedad culinaria.
    aggfunc={"estrellas":np.mean},
    fill_value=0)
agrupacion_culinaria_por_estrellas

estrellas                         
culinaria      Chapina China Italiana Thai Tica
ciudades                                       
Guatemala            3     3        3    5    0
San José             3     2        2    0    4
San Salvador         0     3        2    0    3

In [15]:
agrupacion_culinaria_por_estrellas.plot(kind="bar")

<IPython.core.display.Javascript object>

Entonces, recapitulando:

**¿De qué nos sirven las tablas pivote?**

**¿Que clase de operación representan?**

**¿En que casos podemos usarlas?**

# Agrupando, parte dos

Ya vimos algunos de los criterios básicos de agrupación en el primer webinar, ahora podemos avanzar un poco, combinando agrupación con pivote.

Tambien podemos usar [stack](https://pandas.pydata.org/pandas-docs/stable/reshaping.html), que es otra forma de agrupación basada en índices.

In [16]:
maga_fitosanitario = pd.read_csv("MAGA - CERTIFICADOS FITOSANITARIOS.csv")

In [17]:
hashlib.md5("Hola".encode("UTF-8")).hexdigest()

'f688ae26e9cfa3ba6235477831d5122e'

In [18]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Vamos a limpiar un poco de información
def ofusca_nombre(nombre):
    return humanhash.humanize(hashlib.md5(nombre.encode("UTF-8")).hexdigest())

maga_fitosanitario["Solicitante"] = maga_fitosanitario["Solicitante"].map(ofusca_nombre)
maga_fitosanitario["Fecha Autorización"] = maga_fitosanitario["Fecha Autorización"].map(pd.Timestamp)


In [19]:
def clean_q(input_object):
    from re import sub  ## importamos la función sub, que substituye utilizando patrones
    ## https://es.wikipedia.org/wiki/Expresión_regular
    
    ## NaN es un objeto especial que representa un valor numérico invalido, Not A Number.
    if input_object == NaN:
        return 0
    inp = unicode(input_object) # De objeto a un texto
    cleansed_00 = sub(r'\.000', '000', inp) 
    cleansed_nonchar = sub(r'[^0-9]+', '', cleansed_00)
    if cleansed_nonchar == '':
        return 0
    return cleansed_nonchar

maga_fitosanitario["Kg. Netos"] = maga_fitosanitario["Kg. Netos"].map(clean_q).astype(float)

In [20]:
maga_fitosanitario["Kg. Netos"].head()

0   1360779.0
1    816467.0
2   3483624.0
3   3483624.0
4    151800.0
Name: Kg. Netos, dtype: float64

In [21]:
maga_fitosanitario = pd.read_csv("MAGA - CERTIFICADOS FITOSANITARIOS - LIMPIO.csv")
maga_fitosanitario.head()

,Unnamed: 0,Incidente,Fecha Autorización,Solicitante,Permiso,Producto,Categoría,CIF $,Kg. Netos,Aduana,País procedencia,País origen,Otra fuente de origen
0,0,49,2014-04-04,sodium-pasta-glucose-mexico,142432,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,125.0,1360779.0,MELCHOR DE MENCOS,Belice,Belice,NaN
1,1,50,2014-04-04,sodium-pasta-glucose-mexico,142425,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,75.0,816467.0,MELCHOR DE MENCOS,Belice,Belice,NaN
2,2,275,2014-07-04,robert-vermont-one-robert,142479,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,2584.1,3483624.0,PEDRO DE ALVARADO,Nicaragua,Nicaragua,NaN
3,3,276,2014-07-04,robert-vermont-one-robert,142480,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,2584.1,3483624.0,PEDRO DE ALVARADO,Nicaragua,Nicaragua,NaN
4,4,177,2014-07-04,zebra-missouri-arkansas-island,142478,HARINA DE TRIGO,HARINA DE TRIGO,81601.2,151800.0,TECUN UMAN,México,México,NaN


In [22]:
maga_fitosanitario.groupby("Producto").sum().sort_values("CIF $", ascending=False).head(20)

,Unnamed: 0,Permiso,CIF $,Kg. Netos
Producto,,,,
MAIZ AMARILLO,905380,53491946,183595249.4,2753711966.0
HARINA DE SOYA,561422,18899910,56391529.9,177684115.0
ARROZ EN GRANZA,56017,3201212,24568825.5,59041504.0
HARINA DE TRIGO,1421597,76664602,24272107.3,54953080.0
AJONJOLI NATURAL,186828,9215686,15985707.0,20396870.0
ARROZ GRANZA,238826,15088878,13883822.8,57978262.0
ALGODON,268899,9014988,12028250.9,200973442.0
ALGODON SIN CARDAR NI PEINAR,143934,5222374,8151960.6,19619672.0
MADERA DE PINO (SECA),257598,9285734,6498009.3,11404341.0


In [23]:
maga_productos_pivot = maga_fitosanitario.pivot_table(
    index=["Categoría", "Producto"], 
    values=["CIF $", "Permiso","Kg. Netos"], 
    aggfunc={"CIF $":np.sum,"Permiso":len, "Kg. Netos": np.sum},
    fill_value=0)

In [24]:
maga_productos_pivot

CIF $  \
Categoría               Producto                                                        
AJONJOLI                AJONJOLI DESCORTEZADO                                250093.8   
                        AJONJOLI NATURAL                                   10702707.0   
ALGODÓN                 ALGODON                                            12028250.9   
                        ALGODON (BLANQUEADO)                                  52969.2   
                        ALGODON (EN PACAS)                                    35604.5   
                        ALGODON (PACAS)                                       48087.7   
                        ALGODON (SIN CARDAR NI PEINAR)                      1934207.8   
                        ALGODON (SIN PEINAR NI CARDAR)                       507479.4   
                        ALGODON BLANQUEADO                                    52658.4   
                        ALGODON SI CARDAR NI PEINAR                           45158.1   
                        ALGODON SIN CARDAR NI PEINAR                        8151960.6   
                        ALGODON SIN PEINAR NI CARDAR                        2879479.1   
                        DESPERDICIO DE HILADOS DE ALGODON                     32070.3   
                        PACAS DE ALGODON                                     371093.7   
ALMIDON DE MAIZ         ALMIDON (GLOBE AA SPV/25 KG) DE MAIZ                  12500.0   
                        ALMIDON DE MAIZ                                      487747.5   
                        ALMIDON DE MAIZ (CORRUPAC)                            13515.0   
                        ALMIDON DE MAIZ AA                                     7075.0   
                        ALMIDON DE MAIZ CORRUPAC                              59038.5   
ALPISTE                 ALPISTE                                               54856.6   
ARBOLES NATURALES       SEMILLA DE TECA (TECTONA GRANDIS)                      1388.6   
                        SEMILLAS DE ARBOLES (Lluvia de oro y Papaya)             66.0   
                        SEMILLAS DE ARBOLES (Lluvia de oro, Papaya y St...      127.6   
ARROZ                   ARROZ                                               6111828.4   
                        ARROZ (PARBOIL)                                       97606.0   
                        ARROZ EN GRANO                                        20400.5   
                        ARROZ EN GRANZA                                    21791713.2   
                        ARROZ EN ORO                                          55507.9   
                        ARROZ ESCALDADO                                      240050.0   
                        ARROZ GRANZA                                       13883822.8   
...                                                                               ...   
SUSTRATOS               FIBRA DE COCO                                         78753.3   
                        FIBRA DE COCO (SUSTRATO)                             117332.5   
                        FIBRA DE COCO TRATADA                                132302.4   
                        FIBRA DE ESTOPA DE COCO                                 375.0   
                        PEAT MOSS                                            655093.3   
                        PEAT MOSS (BOLSAS DE ENRAIZADORES)                    16425.0   
                        PEAT MOSS (SUSTRATO )                                  3258.7   
                        PEAT MOSS (SUSTRATO DARK VEGETABLE MIX)                7151.8   
                        PEAT MOSS (SUSTRATO DE TURBA)                         16068.0   
                        PEAT MOSS (SUSTRATO)                                  99434.2   
                        PEAT MOSS (TURBA)                                    213577.5   
                        PEAT MOSS SPHAGNUM (SUSTRATO DE TURBA)                15803.5   
                        PEAT SPHAGNUM                                      

In [25]:
maga_aduanas_pivot = maga_fitosanitario.pivot_table(
    index=["País origen", "Aduana"], 
    values=["CIF $", "Kg. Netos"],
    aggfunc={"CIF $":np.sum,"Kg. Netos":np.mean},
    fill_value=0)
maga_aduanas_pivot

CIF $  Kg. Netos
País origen          Aduana                                       
ALEMANIA             EXPRESS AEREO             102326.9      408.7
                     PUERTO QUETZAL           1727517.6  3000000.0
                     SANTO TOMAS DE CASTILLA    10050.2    16000.0
ARGENTINA            PEDRO DE ALVARADO           3166.1   306817.0
                     PUERTO BARRIOS            163440.7    72900.0
                     PUERTO QUETZAL            187991.7    45965.1
                     SANTO TOMAS DE CASTILLA   410929.5    42113.6
AUSTRALIA            EXPRESS AEREO              94350.0     2775.0
                     PUERTO QUETZAL             26932.2    21375.0
                     SANTO TOMAS DE CASTILLA   113750.0     3464.0
                     TECUN UMAN                  5803.1   546201.0
Alemania             EXPRESS AEREO              29503.2       48.5
Argentina            SANTO TOMAS DE CASTILLA    15510.0    22000.0
BANGLADESH           PUERTO QUETZAL             48274.4    16425.0
                     SANTO TOMAS DE CASTILLA    72000.0   537894.0
BELGICA              EXPRESS AEREO              91348.9     1203.0
                     PUERTO QUETZAL           1855830.6  1500078.0
                     TECUN UMAN                192549.9    65179.2
BELICE               MELCHOR DE MENCOS         547047.4  1097158.3
BOLIVIA              PUERTO QUETZAL             91800.0    18000.0
BRASIL               EXPRESS AEREO             121048.1    11588.5
                     PUERTO BARRIOS            245407.8    17108.0
                     SANTO TOMAS DE CASTILLA  3604767.2   382245.2
BURKINA FASO         SANTO TOMAS DE CASTILLA    42888.0    19500.0
Belice               MELCHOR DE MENCOS          45400.0   578402.5
CANADÁ               EXPRESS AEREO                 72.1      458.0
                     PUERTO BARRIOS            551903.1    51886.2
                     PUERTO QUETZAL            908538.0   175239.1
                     SANTO TOMAS DE CASTILLA  1231395.6    94833.4
CHAD                 PUERTO QUETZAL             35640.0    18000.0
...                                                 ...        ...
PERÚ                 EXPRESS AEREO            2572758.1    12905.7
                     PUERTO QUETZAL            341275.7    14211.7
                     SANTO TOMAS DE CASTILLA    49007.5     5258.3
POLONIA              EXPRESS AEREO               6366.9       30.0
PORTUGAL             EXPRESS AEREO                667.3       12.0
Países Bajos         EXPRESS AEREO               6890.2       40.0
                     SANTO TOMAS DE CASTILLA   586587.1    18211.0
Perú                 EXPRESS AEREO               2428.0      500.0
REINO UNIDO          EXPRESS AEREO                225.2       42.0
REPÚBLICA DOMINICANA SANTO TOMAS DE CASTILLA   734869.1  1859816.9
SRI LANKA            PUERTO QUETZAL            711008.5    52775.5
                     SANTO TOMAS DE CASTILLA   665303.2   381409.6
SUDÁFRICA            EXPRESS AEREO             310826.5   271127.0
SUDÁN                PUERTO QUETZAL             36826.8    12412.5
SUECIA               EXPRESS AEREO              72927.0   392008.0
                     PUERTO QUETZAL           1827487.7  1500000.0
TAILANDIA            EXPRESS AEREO             226567.5    16619.1
                     PUERTO QUETZAL             28765.0     1750.0
TAIWAN               EXPRESS AEREO             205971.6    11966.5
                     VALLE NUEVO                 1200.0       70.0
TANZANIA             PUERTO QUETZAL            330340.0    20000.0
TURQUÍA              EXPRESS AEREO                262.9      145.0
URUGUAY              SANTO TOMAS DE CASTILLA  1485896.5   103641.5
UZBEKISTÁN           SANTO TOMAS DE CASTILLA   106000.0   100240.0
Uruguay              SANTO TOMAS DE CASTILLA    59007.8   108215.0
VENEZUELA            PUERTO QUETZAL          13881240.0   254221.7
                     SANTO TOMAS DE CASTILLA  1185600.0   197600.0
VIETNAM              EXPRESS A

Que tal si indagamos mas en las categorias que se importan de cada país.

In [26]:
maga_aduanas_pivot = maga_fitosanitario.pivot_table(
    index=["País origen", "Aduana", "Categoría"], 
    values=["CIF $"], 
    aggfunc={"CIF $":np.sum},
    fill_value=0)
maga_aduanas_pivot

CIF $
País origen Aduana                  Categoría                               
ALEMANIA    EXPRESS AEREO           ESQUEJES RIZOMAS BULBOS          72024.6
                                    MATERIA PRIMA PARA ALIMENTOS     19900.5
                                    SEMILLAS DE FLORES                 475.8
                                    SEMILLAS DE HORTALIZA             9926.0
            PUERTO QUETZAL          MALTA                          1727517.6
            SANTO TOMAS DE CASTILLA MALTA                            10050.2
ARGENTINA   PEDRO DE ALVARADO       MAIZ PARA CONSUMO HUMANO          3166.1
            PUERTO BARRIOS          FRIJOL NEGRO                    139899.7
                                    FRIJOL PINTO                     23541.0
            PUERTO QUETZAL          FRUTAS                           60811.9
                                    GRANOS                          108570.0
                                    SEMILLAS DE FLORES               18609.9
            SANTO TOMAS DE CASTILLA GRANOS                          171417.0
                                    MAIZ PARA CONSUMO HUMANO         89333.0
                                    MAIZ PARA ELAB DE CONCENTRADOS   34881.5
                                    SEMILLAS DE FLORES               94778.0
                                    SEMILLAS DE HORTALIZA            20520.0
AUSTRALIA   EXPRESS AEREO           SEMILLAS DE HORTALIZA            94350.0
            PUERTO QUETZAL          MADERA CEPILLADA                 26932.2
            SANTO TOMAS DE CASTILLA SEMILLAS DE HORTALIZA           113750.0
            TECUN UMAN              MADERA ASERRADA                   5803.1
Alemania    EXPRESS AEREO           ESQUEJES RIZOMAS BULBOS          26028.2
                                    SEMILLAS DE HORTALIZA             3475.0
Argentina   SANTO TOMAS DE CASTILLA SEMILLAS DE FLORES               15510.0
BANGLADESH  PUERTO QUETZAL          TELA DE YUTE                     48274.4
            SANTO TOMAS DE CASTILLA TELA DE YUTE                     72000.0
BELGICA     EXPRESS AEREO           SEMILLAS DE HORTALIZA            91348.9
            PUERTO QUETZAL          MALTA                          1855830.6
            TECUN UMAN              ALMIDON DE MAIZ                  10500.0
                                    HARINA DE TRIGO                 162327.3
...                                                                      ...
SUDÁN       PUERTO QUETZAL          ESPECIAS                         36826.8
SUECIA      EXPRESS AEREO           SEMILLAS DE HORTALIZA            72927.0
            PUERTO QUETZAL          MALTA                          1827487.7
TAILANDIA   EXPRESS AEREO           ESQUEJES RIZOMAS BULBOS          10223.5
                                    FLORES                            4288.4
                                    SEMILLAS DE FLORES                   5.2
                                    SEMILLAS DE FRUTAS                 611.3
                                    SEMILLAS DE HORTALIZA           210959.6
                                    SEMOLA DE TRIGO                    479.6
            PUERTO QUETZAL          SEMILLAS DE HORTALIZA            28765.0
TAIWAN      EXPRESS AEREO           ESQUEJES RIZOMAS BULBOS           1152.0
                                    FLORES                           27781.0
                                    SEMILLAS DE FRUTAS               80970.4
                                    SEMILLAS DE HORTALIZA            96068.2
            VALLE NUEVO             FLORES                            1200.0
TANZANIA    PUERTO QUETZAL          SEMILLAS DE FLORES              330340.0
TURQUÍA     EXPRESS AEREO           SEMILLAS DE FLORES                 223.7
                                    SEMILLAS DE HORTALIZA               39.2
URUGUAY     SANTO TOMAS DE CASTILLA ARROZ                           463200.0
                                    MADERA ASERRADA                

In [27]:
maga_aduanas_pivot_top10 = maga_aduanas_pivot.sort_values("CIF $", ascending=False).head(10)

In [28]:
maga_aduanas_pivot_top10.plot(kind="barh")

<IPython.core.display.Javascript object>

Tambien es util mostrar la tabla, podemos ponerle un poco de estilo con la funcionalidad de [Seaborn + Pandas](https://pandas.pydata.org/pandas-docs/stable/style.html)

In [29]:
cm_paleta_verde = seaborn.light_palette("green", as_cmap=True)
s = maga_aduanas_pivot_top10.style.background_gradient(cmap=cm_paleta_verde)
s

In [30]:
maga_aduanas_pivot = maga_fitosanitario.pivot_table(
    index=["País origen", "Aduana", "Categoría"], 
    values=["CIF $"], 
    aggfunc={"CIF $":np.sum},
    fill_value=0)
maga_aduanas_pivot
cm_paleta_verde = seaborn.light_palette("green", as_cmap=True)
s = maga_aduanas_pivot_top10.style.background_gradient(cmap=cm_paleta_verde)
s

In [31]:
maga_fitosanitario

,Unnamed: 0,Incidente,Fecha Autorización,Solicitante,Permiso,Producto,Categoría,CIF $,Kg. Netos,Aduana,País procedencia,País origen,Otra fuente de origen
0,0,49,2014-04-04,sodium-pasta-glucose-mexico,142432,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,125.0,1360779.0,MELCHOR DE MENCOS,Belice,Belice,NaN
1,1,50,2014-04-04,sodium-pasta-glucose-mexico,142425,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,75.0,816467.0,MELCHOR DE MENCOS,Belice,Belice,NaN
2,2,275,2014-07-04,robert-vermont-one-robert,142479,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,2584.1,3483624.0,PEDRO DE ALVARADO,Nicaragua,Nicaragua,NaN
3,3,276,2014-07-04,robert-vermont-one-robert,142480,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,2584.1,3483624.0,PEDRO DE ALVARADO,Nicaragua,Nicaragua,NaN
4,4,177,2014-07-04,zebra-missouri-arkansas-island,142478,HARINA DE TRIGO,HARINA DE TRIGO,81601.2,151800.0,TECUN UMAN,México,México,NaN
5,5,152,2014-07-04,beryllium-aspen-colorado-fish,142468,HARINA DE TRIGO,HARINA DE TRIGO,11726.3,20866.0,SAN CRISTOBAL,El Salvador,El Salvador,NaN
6,6,153,2014-07-04,beryllium-aspen-colorado-fish,142467,HARINA DE TRIGO,HARINA DE TRIGO,11726.3,20866.0,SAN CRISTOBAL,El Salvador,El Salvador,NaN
7,7,155,2014-07-04,beryllium-aspen-colorado-fish,142469,HARINA DE TRIGO,HARINA DE TRIGO,11726.3,20866.0,SAN CRISTOBAL,El Salvador,El Salvador,NaN
8,8,156,2014-07-04,beryllium-aspen-colorado-fish,142470,HARINA DE TRIGO,HARINA DE TRIGO,11726.3,20866.0,SAN CRISTOBAL,El Salvador,El Salvador,NaN
9,9,162,2014-07-04,robert-vermont-one-robert,142474,CASCARILLA DE ARROZ,MATERIA PRIMA PARA CONCENTRADOS,2584.1,3483624.0,PEDRO DE ALVARADO,Nicaragua,Nicaragua,NaN


In [35]:
maga_fitosanitario.groupby(["Solicitante","Producto"]).sum().sort_values("CIF $", ascending=False).head(10).style.background_gradient(cmap=cm_paleta_verde)

,,Unnamed: 0,Permiso,CIF $,Kg. Netos
Solicitante,Producto,,,,
papa-cola-cat-utah,MAIZ AMARILLO,28650,1742648,3.4162e+07,2.11388e+09
crazy-december-charlie-leopard,MAIZ AMARILLO,28574,1742518,2.36336e+07,9.22726e+07
blue-alanine-aspen-ink,MAIZ AMARILLO,31428,2029788,2.23108e+07,9.1579e+07
glucose-network-cardinal-december,ARROZ EN GRANZA,23564,1734642,1.99125e+07,4.79402e+07
papa-cola-cat-utah,HARINA DE SOYA,24787,871146,1.74555e+07,2.84362e+07
nineteen-earth-stairway-nebraska,MAIZ AMARILLO,26075,1738702,1.62565e+07,6.60348e+07
fanta-crazy-rugby-salami,AJONJOLI NATURAL,166060,8771752,1.58065e+07,1.99436e+07
butter-ceiling-uncle-salami,MAIZ AMARILLO,10205,864636,1.47784e+07,5.92762e+07
crazy-december-charlie-leopard,HARINA DE SOYA,24813,871130,1.26285e+07,2.05404e+07


In [32]:
maga_fitosanitario.pivot_table(
    index=["Solicitante", "País origen", "Categoría"], 
    values=["CIF $"], 
    aggfunc={"CIF $":np.sum},
    fill_value=0).sort_values("CIF $", ascending=False).head(10)

,,,CIF $
Solicitante,País origen,Categoría,
papa-cola-cat-utah,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,34161962.8
crazy-december-charlie-leopard,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,28967653.8
blue-alanine-aspen-ink,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,20603138.9
papa-cola-cat-utah,ESTADOS UNIDOS,HARINA DE SOYA,17455520.7
glucose-network-cardinal-december,ESTADOS UNIDOS,ARROZ,17135432.4
nineteen-earth-stairway-nebraska,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,16256532.0
butter-ceiling-uncle-salami,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,14778445.5
crazy-december-charlie-leopard,ESTADOS UNIDOS,HARINA DE SOYA,12628533.6
september-red-hamper-ten,ESTADOS UNIDOS,MAIZ PARA ELAB DE CONCENTRADOS,10279335.3


In [33]:
maga_fitosanitario.pivot_table(
    index=["País procedencia"],
    columns=["Aduana"],
    values=["CIF $"], 
    aggfunc={"CIF $":np.sum},
    fill_value=0).style.background_gradient(cmap=cm_paleta_verde)

Que tal si queremos obtener el precio por kilogramo de cada producto y en base a eso obtener los productos mas 'preciosos'.

In [35]:
maga_fitosanitario.to_csv("MAGA - CERTIFICADOS FITOSANITARIOS - LIMPIO.csv")